In [ ]:
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from cbc_paths import file_path_csv, chrome_executable_path

# create selenium driver
#driver = webdriver.Edge(executable_path=edge_executable_path)
driver = webdriver.Chrome(executable_path=chrome_executable_path)
driver.minimize_window()

# read json file data
file = os.path.abspath('') + '\cbc_stories.json'
with open(file) as json_file:
    cbc_stories = json.load(json_file)

print(len(cbc_stories), 'stories')

# loop through stories to get most recent data
story_count = 0
for story in cbc_stories:
    if story['audit_status'] == 'pending':
        story_count += 1
        cbc_url = story['url']
        print(story_count, cbc_url)
        driver.get(cbc_url)
        time.sleep(5)

        # get csv record count aka comment_count_retrieved
        file_name = cbc_url.replace('https://www.cbc.ca/','').replace('news/','').replace('sports/','').replace('/','_') + '.csv'
        try: # new urls may not yet have csv files
            with open(file_path_csv + file_name, encoding='utf-8') as f:
                comment_count_retrieved = str(sum(1 for line in f))
            csv_available = 'yes'
        except: 
            comment_count_retrieved = ''
            csv_available = 'no'

        # get cbc story data from selenium
        try:
            comment_count_site = driver.find_element(By.CLASS_NAME, 'vf-counter').text
        except:
            comment_count_site = ''
        try:
            story_title = driver.find_element(By.CLASS_NAME, 'detailHeadline').text
        except:
            story_title = ''
        try:
            story_date = driver.find_element(By.XPATH, '//time').get_attribute('datetime').split('T')[0]
        except:
            story_date = ''

        # write updated data to json
        story['url'] = cbc_url
        story['story_title'] = story_title
        story['story_date'] = story_date
        story['comment_count_site'] = comment_count_site
        story['comment_count_retrieved'] = comment_count_retrieved
        try:
            story['comment_retrieved_pct'] = int(100 * int(comment_count_retrieved) / int(comment_count_site))
        except:
            story['comment_retrieved_pct'] = '0'
        
        story['csv_available'] = csv_available
        try:
            driver.find_element(By.XPATH, "//*[contains(text(), 'Commenting is now closed for this story')]")
            story['comment_status'] = 'closed'
        except:
            story['comment_status'] = 'open'

# save updated json to json file
with open(file, 'w') as file_object:
    json.dump(cbc_stories, file_object, indent=4)

# close browser
driver.close()